#### Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network

In [1]:
!pip install keras-tuner

In [22]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from kerastuner.tuners import RandomSearch
import IPython

In [3]:
df=pd.read_csv('Real_Combine.csv')

In [4]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [5]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

#### Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate 

In [6]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [7]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Oracle from existing project project/Air Quality Index/oracle.json
INFO:tensorflow:Reloading Tuner from project/Air Quality Index/tuner0.json


In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [24]:
import kerastuner as kt
tuner_band = kt.Hyperband(build_model,
                     objective = 'val_mean_absolute_error', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')


INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


In [25]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [28]:
tuner_band.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner_band.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 42 Complete [00h 00m 03s]
val_mean_absolute_error: 50.61952209472656

Best val_mean_absolute_error So Far: 44.205562591552734
Total elapsed time: 00h 02m 48s
INFO:tensorflow:Oracle triggered exit


ValueError: units does not exist.

In [29]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner_band.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - 0s 9ms/step - loss: nan - mean_absolute_error: nan - val_loss: 62.6099 - val_mean_absolute_error: 62.6099
Epoch 2/10
24/24 [==============================] - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 59.2616 - val_mean_absolute_error: 59.2616
Epoch 3/10
24/24 [==============================] - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 60.5311 - val_mean_absolute_error: 60.5311
Epoch 4/10
24/24 [================

In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 19
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_6 (Int)
{'default': None, 'co

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [10]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 7
units_0: 384
units_1: 224
learning_rate: 0.01
units_2: 256
units_3: 224
units_4: 256
units_5: 64
units_6: 384
units_7: 480
units_8: 256
units_9: 448
units_10: 352
units_11: 224
units_12: 160
units_13: 320
Score: 49.630967458089195
Trial summary
Hyperparameters:
num_layers: 4
units_0: 256
units_1: 416
learning_rate: 0.001
units_2: 320
units_3: 32
units_4: 32
units_5: 128
units_6: 448
units_7: 160
units_8: 256
units_9: 320
units_10: 192
units_11: 256
units_12: 448
units_13: 160
Score: 59.92278544108073
Trial summary
Hyperparameters:
num_layers: 2
units_0: 512
units_1: 256
learning_rate: 0.0001
units_2: 448
units_3: 384
units_4: 320
units_5: 384
units_6: 384
units_7: 96
units_8: 256
units_9: 352
units_10: 416
units_11: 64
units_12: 64
units_13: 64
Score: 62.457140604654946
Trial summary
Hyperparameters:
num_laye

In [30]:
tuner_band.results_summary()

Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 4
units_0: 288
units_1: 224
learning_rate: 0.01
units_2: 64
units_3: 512
units_4: 64
units_5: 288
units_6: 64
units_7: 32
units_8: 320
units_9: 224
units_10: 448
units_11: 160
units_12: 160
units_13: 320
units_14: 448
units_15: 256
units_16: 288
units_17: 352
units_18: 96
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 44.205562591552734
Trial summary
Hyperparameters:
num_layers: 2
units_0: 384
units_1: 448
learning_rate: 0.01
units_2: 288
units_3: 384
units_4: 416
units_5: 128
units_6: 480
units_7: 480
units_8: 128
units_9: 192
units_10: 96
units_11: 256
units_12: 128
units_13: 96
units_14: 320
units_15: 448
units_16: 480
units_17: 192
units_18: 128
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 388720d5b1b16c4bb3fce8f6046b9f0b
Score: 46.60113525